In [134]:
!pip install prophet

     |████████████████████████████████| 65 kB 4.0 MB/s 
     |████████████████████████████████| 49 kB 7.4 MB/s 
     |████████████████████████████████| 45 kB 3.8 MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-py3-none-any.whl size=6639699 sha256=22530745bcb2be4968636d76e8ef63560a7789992c960c6cac6deec41860dc06
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, but you have cmdstanpy 0.9.68 which is incompatible.


In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.patches as mpatches
from plotly.subplots import make_subplots
from wordcloud import WordCloud
from prophet import Prophet
import seaborn as sns  
sns.set(color_codes = True)
sns.set(style="whitegrid")
import plotly.figure_factory as ff
from plotly.colors import n_colors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from plotly.offline import iplot

ModuleNotFoundError: ignored

In [86]:
df = pd.read_csv('country_vaccinations.csv')

In [87]:
df.fillna(value = 0, inplace = True)
df.total_vaccinations = df.total_vaccinations.astype(int)
df.people_vaccinated = df.people_vaccinated.astype(int)
df.people_fully_vaccinated = df.people_fully_vaccinated.astype(int)
df.daily_vaccinations_raw = df.daily_vaccinations_raw.astype(int)
df.daily_vaccinations = df.daily_vaccinations.astype(int)
df.total_vaccinations_per_hundred = df.total_vaccinations_per_hundred.astype(int)
df.people_fully_vaccinated_per_hundred = df.people_fully_vaccinated_per_hundred.astype(int)
df.daily_vaccinations_per_million = df.daily_vaccinations_per_million.astype(int)
df.people_vaccinated_per_hundred = df.people_vaccinated_per_hundred.astype(int)
date = df.date.str.split('-', expand =True)

In [88]:
df['year'] = date[0]
df['month'] = date[1]
df['day'] = date[2]
df.year = pd.to_numeric(df.year)
df.month = pd.to_numeric(df.month)
df.day = pd.to_numeric(df.day)
df.date = pd.to_datetime(df.date)
df = df.drop(['source_name', 'source_website', 'iso_code', 'date', 'year', 'month', 'day', 'vaccines'], axis=1)
df.head()

,country,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,Afghanistan,0,0,0,0,0,0,0,0,0
1,Afghanistan,0,0,0,0,1367,0,0,0,34
2,Afghanistan,0,0,0,0,1367,0,0,0,34
3,Afghanistan,0,0,0,0,1367,0,0,0,34
4,Afghanistan,0,0,0,0,1367,0,0,0,34


# Encoding categorical values

In [89]:
def standarscaler(data):
  scaler = preprocessing.StandardScaler()
  train_col_scale = data[['total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'daily_vaccinations_raw', 'total_vaccinations_per_hundred',
                          'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'daily_vaccinations_per_million']]
  train_scaler_col = scaler.fit_transform(train_col_scale)
  train_scaler_col = pd.DataFrame(train_scaler_col, columns=train_col_scale.columns)
  data['total_vaccinations'] = train_scaler_col['total_vaccinations']
  data['people_vaccinated'] = train_scaler_col['people_vaccinated']
  data['people_fully_vaccinated'] = train_scaler_col['people_fully_vaccinated']
  data['daily_vaccinations_raw'] = train_scaler_col['daily_vaccinations_raw']
  #data['daily_vaccinations'] = train_scaler_col['daily_vaccinations']
  data['total_vaccinations_per_hundred'] = train_scaler_col['total_vaccinations_per_hundred']
  data['people_vaccinated_per_hundred'] = train_scaler_col['people_vaccinated_per_hundred']
  data['people_fully_vaccinated_per_hundred'] = train_scaler_col['people_fully_vaccinated_per_hundred']
  data['daily_vaccinations_per_million'] = train_scaler_col['daily_vaccinations_per_million']
  return data

In [90]:
def onehotencoder(dataset):
  le = preprocessing.LabelEncoder() # label encoder 
  dataset['country'] = le.fit_transform(dataset['country']) 
  dataset = pd.get_dummies(dataset,drop_first=True)
  return dataset

In [91]:
dataframe = onehotencoder(df)
dataframe = standarscaler(df)
dataframe

,country,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,0,-0.149599,-0.147388,-0.115274,-0.145568,0,-0.633945,-0.656754,-0.599044,-0.767446
1,0,-0.149599,-0.147388,-0.115274,-0.145568,1367,-0.633945,-0.656754,-0.599044,-0.759603
2,0,-0.149599,-0.147388,-0.115274,-0.145568,1367,-0.633945,-0.656754,-0.599044,-0.759603
3,0,-0.149599,-0.147388,-0.115274,-0.145568,1367,-0.633945,-0.656754,-0.599044,-0.759603
4,0,-0.149599,-0.147388,-0.115274,-0.145568,1367,-0.633945,-0.656754,-0.599044,-0.759603
...,...,...,...,...,...,...,...,...,...,...
21476,55,-0.149599,-0.147388,-0.115274,-0.145568,608,-0.633945,-0.656754,-0.599044,-0.627429
21477,55,-0.149599,-0.147388,-0.115274,-0.145568,608,-0.633945,-0.656754,-0.599044,-0.627429
21478,55,-0.149599,-0.147388,-0.115274,-0.145568,608,-0.633945,-0.656754,-0.599044,-0.627429
21479,55,-0.149599,-0.147388,-0.115274,-0.145568,608,-0.633945,-0.656754,-0.599044,-0.627429


# Data spliting

In [92]:
# Function to split the dataset
def splitdataset(data):
  X = data.drop(['daily_vaccinations'],axis=1)
  Y = data['daily_vaccinations']
  # Splitting the dataset into train and test
  X_train, X_test, y_train, y_test = train_test_split( 
  X, Y, test_size = 0.3, random_state = 100)
  return X, Y, X_train, X_test, y_train, y_test

In [113]:
X, Y, X_train, X_test, y_train, y_test = splitdataset(dataframe)

# Desition Trees

In [105]:
# Function to calculate accuracy
def accuracy_tree(clf_entropy, y_test_tree, y_pred_tree):
  cross_val_score(clf_entropy, X_train, y_train, cv=10)
  print("Confusion Matrix: ", '\n',
        confusion_matrix(y_test_tree, y_pred_tree))
  print("Accuracy: ",
        accuracy_score(y_test_tree,y_pred_tree)*100)
  print("Report: ", '\n',
        classification_report(y_test, y_pred_tree))

In [6]:
# Function to make predictions
def prediction_tree(X_test_tree, clf_object_tree):
  # Prediction on test with giniIndex
  y_pred_tree = clf_object_tree.predict(X_test_tree)
  print("Predicted values:")
  print(y_pred_tree)
  return y_pred_tree

In [7]:
# Function to perform training with entropy.
def train_tree(X_train_tree, y_train_tree, depth, leaf):
  # Decision tree with entropy
  clf_entropy = DecisionTreeClassifier(
    criterion = "entropy", random_state = 100,
    max_depth = depth, min_samples_leaf = leaf)
  # Performing training
  clf_entropy.fit(X_train_tree, y_train_tree)
  return clf_entropy

In [106]:
# Tree Number One
clf_entropy_1 = train_tree(X_train, y_train, 15, 20)
print("Results Using Entropy with depth of 3 and min leaf of 5:")
y_pred_entropy = prediction_tree(X_test, clf_entropy_1)
accuracy_tree(clf_entropy_1, y_test, y_pred_entropy)

Results Using Entropy with depth of 3 and min leaf of 5:
Predicted values:
[1202726      27    1698 ... 8455714   12317   18894]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=10.



Confusion Matrix:  
 [[54  0  0 ...  0  0  0]
 [ 1  4  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
Accuracy:  17.43987587276959
Report:  
               precision    recall  f1-score   support

           0       0.95      1.00      0.97        54
           1       0.33      0.80      0.47         5
           2       0.00      0.00      0.00         3
           3       0.44      0.89      0.59         9
           4       0.00      0.00      0.00         4
           5       0.22      0.33      0.27         6
           6       0.53      0.83      0.65        12
           7       0.22      0.40      0.29         5
           8       0.12      0.75      0.21         4
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         2
          11       0.40      0.40      0.40         5
          12       0.00      0.00      0.00         5
          13       0.00   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control 

In [107]:
# Tree Number Two
clf_entropy_2 = train_tree(X_train, y_train, 30, 40)
print("Results Using Entropy with depth of 3 and min leaf of 5:")
y_pred_entropy = prediction_tree(X_test, clf_entropy_2)
accuracy_tree(clf_entropy_2, y_test, y_pred_entropy)

Results Using Entropy with depth of 3 and min leaf of 5:
Predicted values:
[260372     27   1244 ... 141889  30781  10051]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=10.



Confusion Matrix:  
 [[54  0  0 ...  0  0  0]
 [ 1  4  0 ...  0  0  0]
 [ 1  1  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
Accuracy:  14.026377036462373
Report:  
               precision    recall  f1-score   support

           0       0.95      1.00      0.97        54
           1       0.16      0.80      0.27         5
           2       0.00      0.00      0.00         3
           3       0.13      0.89      0.23         9
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         6
           6       0.37      0.83      0.51        12
           7       0.00      0.00      0.00         5
           8       0.21      0.75      0.33         4
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         2
          11       0.12      0.40      0.19         5
          12       0.00      0.00      0.00         5
          14       0.27  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control 

In [108]:
# Tree Number Three
clf_entropy_3 = train_tree(X_train, y_train, 60, 80)
print("Results Using Entropy with depth of 3 and min leaf of 5:")
y_pred_entropy = prediction_tree(X_test, clf_entropy_3)
accuracy_tree(clf_entropy_3, y_test, y_pred_entropy)

Results Using Entropy with depth of 3 and min leaf of 5:
Predicted values:
[ 260372      27    4307 ... 2221000   11815   11815]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=10.



Confusion Matrix:  
 [[54  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
Accuracy:  10.892164468580296
Report:  
               precision    recall  f1-score   support

           0       0.68      1.00      0.81        54
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         9
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         6
           6       0.26      0.83      0.40        12
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         4
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         2
          11       0.00      0.00      0.00         5
          12       0.00      0.00      0.00         5
          14       0.11  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control 

# Lineal regresion

In [128]:
X_test.reset_index()

,index,country,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,10823,27,0.253854,2.026131,0.551660,0.546375,0.182071,0.715834,-0.061837,0.645640
1,2160,5,-0.149599,-0.147388,-0.115274,-0.145568,-0.633945,-0.656754,-0.599044,-0.310027
2,13193,34,-0.149599,-0.147388,-0.115274,-0.145568,-0.633945,-0.656754,-0.599044,-0.742303
3,9478,23,-0.145693,-0.126683,-0.109028,-0.138793,-0.495376,-0.450866,-0.522300,-0.472880
4,4978,12,-0.143407,-0.121602,-0.099271,-0.129934,-0.372204,-0.347922,-0.330440,-0.268967
...,...,...,...,...,...,...,...,...,...,...
6440,8284,20,-0.149599,-0.147388,-0.115274,-0.145568,-0.633945,-0.656754,-0.599044,-0.753375
6441,6847,16,-0.148571,-0.143503,-0.112273,-0.145555,0.905709,1.161925,1.204436,-0.475648
6442,16140,41,9.961479,-0.147388,-0.115274,5.305297,2.399173,-0.656754,-0.599044,0.906298
6443,7694,18,-0.089688,0.071561,0.071032,-0.123668,1.598553,1.882534,2.163734,-0.281193


In [129]:
y_test.reset_index()

,index,daily_vaccinations
0,10823,1310848
1,2160,30
2,13193,2958
3,9478,15111
4,4978,22092
...,...,...
6440,8284,761
6441,6847,364
6442,16140,10479429
6443,7694,24522


In [132]:
model = LinearRegression().fit(X_train, y_train)
# Obtain the coefficient of determination by calling the model with the score() function, then print the coefficient:
r_sq = model.score(X_train, y_train)
print('coefficient of determination:', r_sq)

# Print the Intercept:
print('intercept:', model.intercept_)

# Print the Slope:
print('slope:', model.coef_) 

# Predict a Response and print it:
y_pred = model.predict(X_test)
print('Predicted response:', y_pred, sep='\n')

coefficient of determination: 0.9592800447422799
intercept: 214327.07431017878
slope: [-1.04019810e+02  7.72288273e+04 -4.32436015e+03 -1.98061675e+04
  1.25214622e+06 -6.60575082e+03 -2.60744864e+04  2.04196524e+04
  2.64181054e+04]
Predicted response:
[ 891503.18672475   23791.55408655    9355.06490415 ... 7638290.89986418
   26070.28452408   52712.61012924]
